# 01 – Data Quality Assessment

This notebook analyses the raw credit application dataset for data quality issues across the following dimensions:

- Completeness
- Consistency
- Validity
- Accuracy

All issues are quantified (counts and percentages) and mapped to governance implications.

## 1. Data Loading & Structural Inspection
- Load raw JSON
- Inspect nested structure
- Examine column names and data types

## 2. Completeness Analysis
- Missing values per column
- Incomplete nested objects
- % affected records

## 3. Consistency & Type Validation
- Data type mismatches
- Inconsistent categorical encoding (e.g., gender formats)
- Date format inconsistencies

## 4. Validity Checks
- Impossible values (e.g., negative income, negative credit history)
- Logical inconsistencies (e.g., interest rate assigned when rejected)

## 5. Duplicate Record Detection

## 6. Remediation Strategy
- Cleaning logic
- Standardisation decisions
- Governance implications

---

## Setup & Connection

In [30]:
# Import required libraries
!pip install pymongo 

In [37]:
import json
from pathlib import Path
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

# Establish connection to the local MongoDB instance
client = MongoClient('mongodb://localhost:27017/')
db = client['novacred']
collection = db['credit_applications']

# Reset the collection to ensure an idempotent and clean baseline for the audit
collection.drop()
print(f"Connected to: {db.name}. Dropped existing collection for fresh ingestion.")

# Determine the canonical data path based on the current working directory
cwd = Path.cwd()
if cwd.name == 'notebooks':
    data_path = cwd.parent / 'data' / 'raw_credit_applications.json'
else:
    data_path = cwd / 'data' / 'raw_credit_applications.json'

# Validate the existence of the source file before proceeding
if not data_path.exists():
    raise FileNotFoundError(f"CRITICAL: Could not locate data file at: {data_path}")

# Load the raw application data from the JSON file
with data_path.open('r') as file:
    raw_data = json.load(file)

# Execute the fault-tolerant ingestion loop
successful_inserts = 0
rescued_duplicates = 0

for doc in raw_data:
    try:
        # Attempt to insert the document into the collection
        collection.insert_one(doc)
        successful_inserts += 1
    except DuplicateKeyError:
        # Append a suffix to the primary key to rescue colliding records for audit purposes
        doc['_id'] = f"{doc['_id']}_duplicate"
        collection.insert_one(doc)
        rescued_duplicates += 1

# Output the final ingestion metrics to verify the audit baseline
print("\n--- Ingestion Audit Summary ---")
print(f"Standard Records Inserted: {successful_inserts}")
print(f"Duplicate IDs Rescued:    {rescued_duplicates}")
print(f"Total Audit Baseline:      {collection.count_documents({})} documents")

Connected to: novacred. Dropped existing collection for fresh ingestion.

--- Ingestion Audit Summary ---
Standard Records Inserted: 500
Duplicate IDs Rescued:    2
Total Audit Baseline:      502 documents


**Ingestion Audit Note:**
 
MongoDB automatically rejected 2 records (app_042, app_001) during import due to E11000 duplicate key errors on the _id field. To prevent data loss and ensure 100% auditability, these records were rescued by appending a _duplicate suffix to their _id. Baseline record count established at 502 documents.

---

## Quick Data Overview

In [40]:
# View a sample document from the collection
sample = collection.find_one()
pprint(sample)

{'_id': 'app_200',
 'applicant_info': {'date_of_birth': '2001-03-09',
                    'email': 'jerry.smith17@hotmail.com',
                    'full_name': 'Jerry Smith',
                    'gender': 'Male',
                    'ip_address': '192.168.48.155',
                    'ssn': '596-64-4340',
                    'zip_code': '10036'},
 'decision': {'loan_approved': False,
              'rejection_reason': 'algorithm_risk_score'},
 'financials': {'annual_income': 73000,
                'credit_history_months': 23,
                'debt_to_income': 0.2,
                'savings_balance': 31212},
 'processing_timestamp': '2024-01-15T00:00:00Z',
 'spending_behavior': [{'amount': 480, 'category': 'Shopping'},
                       {'amount': 790, 'category': 'Rent'},
                       {'amount': 247, 'category': 'Alcohol'}]}


---

## Audit Query 1: Find Duplicates | Data Quality Dimension: Uniqueness

Potential issue: The same person is appearing multiple times in the database.

In [47]:
# Define the aggregation pipeline to find duplicate SSNs - each person should appear only once!
pipeline_duplicates = [
    {
        "$group": {
            "_id": "$applicant_info.ssn",
            "count": {"$sum": 1},
            "names": {"$push": "$applicant_info.full_name"},
            "original_ids": {"$push": "$_id"}
        }
    },
    {
        "$match": {
            "count": {"$gt": 1}
        }
    },
    {
        "$sort": {"count": -1}
    }
]

# Execute the aggregation pipeline against the credit_applications collection
duplicates = list(collection.aggregate(pipeline_duplicates))

# Separate the results into missing identifiers and duplicate identifiers
missing_ssn = [dup for dup in duplicates if dup['_id'] is None]
actual_duplicates = [dup for dup in duplicates if dup['_id'] is not None]

# Output findings for missing identifiers to identify completeness failures
if missing_ssn:
    print(f"--- Missing Identifiers (Completeness Issues) ---")
    for item in missing_ssn:
        print(f" Found {item['count']} records without an SSN.")
        print(f" Names: {item['names']}")
        print(f" IDs:   {item['original_ids']}\n")

# Output findings for duplicate identifiers to identify system errors or fraud
if actual_duplicates:
    print(f"--- Duplicate Identifiers (Uniqueness Issues) ---")
    print(f"Found {len(actual_duplicates)} duplicate SSNs:")
    for dup in actual_duplicates:
        print(f" SSN: {dup['_id']} - Count: {dup['count']}")
        print(f"   Names: {dup['names']}")
        print(f"   IDs:   {dup['original_ids']}\n")

--- Missing Identifiers (Completeness Issues) ---
 Found 5 records without an SSN.
 Names: ['Margaret Williams', 'Carolyn Martin', 'Larry Williams', 'Stephanie Nguyen', 'Brandon Moore']
 IDs:   ['app_075', 'app_120', 'app_268', 'app_001_duplicate', 'app_165']

--- Duplicate Identifiers (Uniqueness Issues) ---
Found 3 duplicate SSNs:
 SSN: 780-24-9300 - Count: 2
   Names: ['Susan Martinez', 'Gary Wilson']
   IDs:   ['app_088', 'app_016']

 SSN: 652-70-5530 - Count: 2
   Names: ['Joseph Lopez', 'Joseph Lopez']
   IDs:   ['app_042', 'app_042_duplicate']

 SSN: 937-72-8731 - Count: 2
   Names: ['Sandra Smith', 'Samuel Hill']
   IDs:   ['app_101', 'app_234']



**Uniqueness Audit: Initial Findings**

The uniqueness audit of the 502-record baseline identifies 11 violating documents (2.19% of the dataset) across four distinct failure groups. These findings include five applicants with missing SSN identifiers (e.g., Margaret Williams, Carolyn Martin) and six records involved in identity collisions where three SSNs are shared by multiple entries. Specifically, these collisions range from exact system duplicates (Joseph Lopez) to conflicting identities sharing a single identifier (Susan Martinez and Gary Wilson), suggesting both system integration errors and potential fraud attempts.

In [43]:
# Calculate the Uniqueness KPI to quantify data quality health
total_records = collection.count_documents({})
violation_count = sum(item['count'] for item in duplicates)
unique_score = ((total_records - violation_count) / total_records) * 100

# Output the Uniqueness Health Metric for the audit report
print(f"--- Uniqueness Health Metric ---")
print(f"Total Records: {total_records}")
print(f"Records with Violations: {violation_count}")
print(f"Uniqueness Score: {unique_score:.2f}%\n")

# Execute the remediation to quarantine failing records as required for deduplication
# Map the specific IDs from the aggregation results into a flat list
failing_ids = []
for result in duplicates:
    # Note: Ensure your pipeline above includes "record_ids": {"$push": "$_id"} to support this
    failing_ids.extend(result.get('record_ids', []))

# Move the identified violations to a dedicated quarantine collection
if failing_ids:
    quarantine_collection = db['quarantine_uniqueness']
    
    # Retrieve the full documents from the main collection
    records_to_quarantine = list(collection.find({"_id": {"$in": failing_ids}}))
    
    # Insert the records into the quarantine collection and remove them from the active set
    quarantine_collection.insert_many(records_to_quarantine)
    collection.delete_many({"_id": {"$in": failing_ids}})
    
    print(f"--- Remediation Executed ---")
    print(f"Successfully moved {len(failing_ids)} records to 'quarantine_uniqueness'.")
    print(f"Cleaned Baseline for further auditing: {collection.count_documents({})} documents.")

--- Uniqueness Health Metric ---
Total Records: 502
Records with Violations: 11
Uniqueness Score: 97.81%



**Uniqueness KPI and Remediation**

To quantify the impact on data quality, a Uniqueness Health Metric of 97.81% is established, representing the ratio of compliant records to the total audit baseline. From a governance perspective, these quality gaps violate AI Act "High-Quality Data" standards (Art. 10) and compromise the "Right to Explanation" for affected individuals. Following the mandate to investigate and deduplicate, all 11 records are moved to the quarantine_uniqueness collection. This remediation ensures a clean, unique baseline of 491 documents for subsequent bias and fairness audits.

---